In [ ]:
import psi4
import psiresp
from rdkit import Chem
from rdkit.Chem import AllChem
import pandas as pd
import os

In [ ]:
smiles_string = 'CNC([C@@H](NC(C)=O)CSCc1ccccc1)=O'

In [ ]:
mol = Chem.MolFromSmiles(smiles_string)
mol = Chem.AddHs(mol)
mol

In [ ]:
for i, atom in enumerate(mol.GetAtoms()):
    atom.SetProp("molAtomMapNumber", str(atom.GetIdx()))

mol

In [ ]:
n_confs = 50

confs = AllChem.EmbedMultipleConfs(mol, numConfs=n_confs, randomSeed=-1)
AllChem.AlignMolConformers(mol)
AllChem.MMFFOptimizeMoleculeConfs(mol, maxIters=1000)

In [ ]:
psirespmol = psiresp.Molecule.from_rdkit(mol)
psirespmol

In [ ]:
psirespmol.optimize_geometry=True

constraints = psiresp.ChargeConstraintOptions(symmetric_atoms_are_equivalent=True)

In [ ]:
geometry_options = psiresp.QMGeometryOptimizationOptions(method="hf", basis="6-31g*")

esp_options = psiresp.QMEnergyOptions(method="hf", basis="6-31g*")

In [ ]:
job_resp = psiresp.Job(molecules=[psirespmol],
            charge_constraints=constraints,
            qm_optimization_options=geometry_options,
            qm_esp_options=esp_options,
            working_directory="resp_charges_generation")

In [ ]:
job_resp.generate_conformers()
job_resp.generate_orientations()
try:
    job_resp.run(update_molecules=True)
except:
    pass

In [ ]:
os.chdir("resp_charges_generation/optimization/")
os.system("bash run_optimization.sh")
os.chdir("../../")
try:
    job_resp.run()
except:
    pass

In [ ]:
os.chdir("resp_charges_generation/single_point/")
os.system("bash run_single_point.sh")
os.chdir("../../")
try:
    job_resp.run()
except:
    pass

In [ ]:
molecule = job_resp.molecules[0]
molecule.charges

In [ ]:
charges_dataframe = pd.DataFrame(molecule.charges)
charges_dataframe.to_csv('fragment_resp_charges.csv')